
## *Pitch detection over time*

- [Aubio](https://aubio.org/manual/latest/)
- [Aubio pitch methods](https://aubio.org/manpages/latest/aubiopitch.1.html)


In [ ]:

import sys
import numpy as np
from numpy import ma
from aubio import source, pitch
from matplotlib import pyplot as plt

%matplotlib inline


In [ ]:

filename = "09_Creeley-Robert_23_The-Warning_Chicago_5-15-61.wav"


downsample = 1
samplerate = 44100 // downsample

win_s = 2048 // downsample # fft size
hop_s = 512  // downsample # hop size

s = source(filename, samplerate, hop_s)
samplerate = s.samplerate

tolerance = 0.8

pitch_o = pitch("yinfft", win_s, hop_s, samplerate)
pitch_o.set_unit("Hz")
pitch_o.set_tolerance(tolerance)

pitches = []
confidences = []


In [ ]:

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    pitch = pitch_o(samples)[0]
    #pitch = int(round(pitch))
    confidence = pitch_o.get_confidence()
    #print("%f %f %f" % (total_frames / float(samplerate), pitch, confidence))
    pitches += [pitch]
    confidences += [confidence]
    total_frames += read
    if read < hop_s: break

plt.figure(figsize=(100,5))
plt.plot(pitches)

plt.show()


In [ ]:

pitches = np.array(pitches)
confidences = np.array(confidences)

cleaned_pitches = ma.masked_where(confidences < tolerance, pitches)
cleaned_pitches = ma.masked_where(cleaned_pitches > 1000, cleaned_pitches)

plt.figure(figsize=(100,5))
plt.plot(cleaned_pitches)
plt.show()


In [ ]:

np.mean(cleaned_pitches)

In [ ]:

plt.figure(figsize=(20,10))
plt.hist(cleaned_pitches)